<a href="https://colab.research.google.com/github/castillosebastian/NLU_legal_domain/blob/master/NLU_textos_legales_primeros_pasos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploracion Fallos Jurisprudencia SCBA

In [2]:
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)

drive.mount(ROOT)           # we mount the google drive at /content/drive

/content/drive
Mounted at /content/drive


In [20]:
%cd  'My Drive/Colab Notebooks/data'

/content/drive/My Drive/Colab Notebooks/data


In [21]:
%ls

corpus_fallosmetdat.json  tbdoctrina.json  tbmetdat.json
corpus_textosfallos.json  tbfallos.json
